In [1]:
import cv2
import argparse
import numpy as np
from skimage.feature import graycomatrix
from skimage.measure import shannon_entropy

In [2]:
def real_time_segmentation(method):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        # Your real-time segmentation logic goes here
        if method == "edge":
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame = cv2.Canny(frame, 50, 150)
        elif method == "color":
            # Example color range for blue
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
            lower_color = np.array([30, 50, 50])
            upper_color = np.array([60, 255, 255])
            mask = cv2.inRange(hsv, lower_color, upper_color)
            frame = cv2.bitwise_and(frame, frame, mask=mask)
        elif method == "texture":
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            distances = [1]
            angles = [0]
            glcm = graycomatrix(gray, distances=distances, angles=angles, symmetric=True, normed=True)
            contrast = glcm[0, 0, 0, 0] 

            # Consider using the grayscale image instead of the single contrast value
            _, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            mask = np.where(gray > threshold, 255, 0).astype(np.uint8)
            mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
            mask = mask.astype(frame.dtype)

            frame = cv2.bitwise_and(frame, frame, mask=mask)
        elif method == "intensity":
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, thresholded = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
            frame = thresholded
        elif method == "contour":
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            _, thresholded = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY)
            contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            frame = frame.copy()
            cv2.drawContours(frame, contours, -1, (0, 255, 0), 2)

        cv2.imshow(f'Real-Time {method.capitalize()} Segmentation', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [7]:
real_time_segmentation("color")